<a href="https://colab.research.google.com/github/thecrazyphysicist369/Amnesic-Flooding/blob/main/Amnesic_Flooding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This is the code to test the consequences of Amnesic Flooding in a Wattz-Strogatz Small World Network.

A WS-Small world network is created with user input of:


*   **n** = *Number of individuals in the population. This is similar to the number of nodes in the small world network*.
*   **k** = *Number of nearest-neighbours. The number of nodes connected to each node in the small world network.*
*   **p** = *Probability of the length of connection between the nodes of the small world network.*


In [ ]:
#Importing all the necessary libraries
import matplotlib.pyplot as plt
import networkx as nx
import random
import numpy as np
import time
import math
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import time

### Generating the Population


In [ ]:
#vertex generator generates each individual with their properties
def gen_vertex(n,percent):
    ver_list=[]
    c=0

    for i in range(n):
      vertex = {
                'index': c, 			# the unique index of the voter
                'connected': [] ,  # the small world connections between the individuals
                'memory': [], 
                'generator' : random.randint(0,percent) ,
                'msg_sharability' : random.randint(0,10),
                'node_sharability' : 0,
                'mymsg':'0',
                'cur_msg':[]
                }
      ver_list.append(vertex)
      c+=1
    return ver_list

#Generating the Small World Network
def gen_graph(n, k, p,percent):
   population = gen_vertex(n,percent)
   G = nx.watts_strogatz_graph(n, k, p, seed=10)
   neighbor = []
   for node in list(sorted(G.nodes())):
   	neighbours = list(nx.all_neighbors(G, node))
   	neighbor.append(neighbours)
   for i in range(n):
     population[i]['connected'] = neighbor[i]
     population[i]['index'] = int(i)
     population[i]['node_sharability'] = random.randint(0,k)
   return population



###Creating the list of possible messages

In [ ]:
 '''
this function generates and returns a list of msgs of strings. 

'''
def msgList():
  list1=[]
  for i in range(60,123) :
    for j in range(60,123) :
      p=chr(i)+chr(j)
      list1.append(p)
  return list1


### Generation of messages by few nodes

In [ ]:
'''
this function randomly chooses from the list of msg and assigns it to one of the nodes that are generators 
'''
def  msgGeneration(population):
  list1=msgList()
  for i in population:
    if(i['generator']==1 ):
      l=random.randint(0,len(list1))
      i['mymsg']=list1[l]
      list1.pop(l)
  #print(len(list1))


#population



### Distribution of messages all over the graph

In [ ]:
'''
This function is for passing the information.
It takes one vertex at a time
It checks if the vertex's neighbors has the same message as the vertex.
If it doesn't have, the vertex sends the message to the neighbor.
'''
def roundcalc_Unique_msg(population,k):
  nxt=[]
  cur=[]
  for node in population:
    if(node['generator']==1 ):
      #print(node['index'])
      nxt.append(node['index'])
      node['cur_msg'].append(node['mymsg'])
  
  
  round=0
  while(True):
    #print(nxt)
    if(len(nxt)==0):
      break
    cur=nxt
    nxt=[]
    
    round=round+1
    for el in cur:
      for adj in population[el]['connected']:
        flag=0
        for msg in population[el]['cur_msg']:
          msg=msg[:2]+str(el)
          #print(msg)
          if(msg in population[adj]['memory']):
            #print("pass"," ",msg," ",el," ",adj)
            continue
          if(len(population[adj]['memory'])>=k):
            flag=1
            population[adj]['memory'].pop(0)
            population[adj]['cur_msg'].append(msg)
            #print(el," ",adj," ",population[adj]['cur_msg']," memm---",population[adj]['memory'])
            population[adj]['memory'].append(msg)
          else:
            flag=1
            population[adj]['cur_msg'].append(msg)
            population[adj]['memory'].append(msg)
           # print("gyyygyg--- ",el," ",adj," ",population[adj]['cur_msg']," memm---",population[adj]['memory'])

        if(flag==1):
          if(adj not in nxt):
            nxt.append(adj)
      population[el]['cur_msg']=[]
      
      #print(round)
  #print(round,end=" ")
  return round







###Nuber of Rounds Vs Memory Size

###The Main Function


This function plots the output but doesn't save the data in a .csv file

In [ ]:

def user_input(no_nodes,no_of_connected_components,p,percent):
  population = gen_graph(no_nodes, no_of_connected_components, p,percent)
  g=0
  x=[] ##g*n-1,g*n
  y=[]
  m=0
  s=0
  for node in population:
    if(node['generator']==1):
      g=g+1
  msgGeneration(population)
  print(g)
  for k in range(g*(no_of_connected_components-1),g*(no_of_connected_components+2)):
    for node in population:
      node['cur_msg']=[]
      node['memory']=[]
    population1=population
    r=roundcalc_Unique_msg(population,k)
    x.append(k)
    y.append(r)
  x_train=np.array(x)
  y_train=np.array(y)
  print(g)
  plt.plot(x_train, y_train)
  plt.xlabel("memory_assigned_to_each_node")
  plt.ylabel("no_of_rounds")
  plt.show()
#population


This function saves the output as a .csv file along with plotting the output in a graph

In [ ]:
##User_input
def simulation(no_nodes,no_of_connected_components,p,percent,obs_no):
  population = gen_graph(no_nodes, no_of_connected_components, p,percent)
  g=0
  x=[] ##g*n-1,g*n
  y=[]
  for node in population:
    if(node['generator']==1):
      g=g+1
  msgGeneration(population)
  #print(g)
  time_elapsed = 0
  #for k in range(g*(no_of_connected_components-1),g*(no_of_connected_components+3)):
  low = g*(no_of_connected_components-1)
  high = g*(no_of_connected_components+3)
  for k in range(low, high):
    tic = time.time()
    for node in population:
      node['cur_msg']=[]
      node['memory']=[]
    population1=population
    r=roundcalc_Unique_msg(population,k)
    x.append(k)
    y.append(r)
    toc = time.time()
    time_elapsed = time_elapsed + math.ceil(toc-tic)
    time_remaining = (high-k)*math.ceil(toc-tic)
    print("Executing Iteration: ", k-low, " Time taken: ", math.ceil(toc-tic),"s ", "Elapsed time: ", time_elapsed, "s Est. Time remaining: ", time_remaining,"s")
  x_train=np.array(x)
  y_train=np.array(y)
  #print(g)
  plt.plot(x_train, y_train)
  plt.xlabel("memory_assigned_to_each_node")
  plt.ylabel("no_of_rounds")
  plt.show()
  no_connected=[]
  no_of_generators=[]
  dist_probability=[]
  for j in x_train:
    no_connected.append(no_of_connected_components)
    no_of_generators.append(g)
    dist_probability.append(p)
  frame=np.array([x_train,y_train,no_connected,no_of_generators,dist_probability])
  dataset=pd.DataFrame(frame)
  dataset=dataset.T
  dataset.columns=["memory_assigned_to_each_node","no_of_rounds","no_connected_components","no_of_generator","distance_prob"]
  dataset.to_csv("observation"+obs_no+".csv")
#population



In [ ]:
pop = int(input("The number of nodes the small world network (population) has : "))
neighbors = int(input("The number of nearest-neighbors for each nodes : "))
prob =  float(input("The probability of length of the edges : "))
generators = int(input("The percent of nodes in the population those who generate content : "))
name = input("A unique name to store your .csv file. WARNING: Same name can overwrite files permanently : ")
simulation(pop, neighbors, prob, generators, name)
#user_input(250,4,0.5,9,"2")

KeyboardInterrupt: ignored

In [ ]:
#make the number of messages as user input
#make the memory length as user input
#change the incoming tagged messages into just incoming messages. Remove the tag of who is sending the messages
#also store the forward number of unique messages vs iteration
    ## aa forwarded how many times before it becomes irrelevant
    ## ab forwarded how m
#length of memory and number of unique messages
    ##quadratic
    ##exponential
    ##linear
    ##<linear